
# 🚀  Applied Deep Learning

**Topic:** Create a search enigne to look up clothes on Zalando using SBERT and semantic search. 

Things we can do: 
- Use images and CLIP
- SetFit for supervised tasks with SBERT models 

Inspo: 
- GIF search engine
- Youtube Search 
- get podcasts transcript for a specific topic (or create transcripts with Whisper - search for OpenAI Whisper Colab)
- Finetune an SBERT model using domain adaptation 
- Embed and build a search engine 
- Build a Gradio App

Nilso's idea: 
- downlod pictures from zalando and then encode them
- And then the user can query, write a string which we encode and then we compare the cos sim distances between the pictures and show them ordered
-> How the fuck do we scrape from Zalando + the search in Zalando is not too shitty. 


theory + exemple: 
https://www.sbert.net/examples/applications/image-search/README.html
https://www.pinecone.io/learn/gif-search/
https://medium.com/mlearning-ai/semantic-search-with-s-bert-is-all-you-need-951bc710e160
https://www.youtube.com/watch?v=6yPWtdgs5Sg&ab_channel=code_your_own_AI
https://github.com/sonynka/fashion_scraper


## Import dependencies  & load dataset

In [ ]:

from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import torch
import pickle
import zipfile
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm.autonotebook import tqdm
torch.set_num_threads(4)


## Initializing Components
CLIP (Contrastive Language-Image Pre-Training) is a neural network trained on a variety of (image, text) pairs. It can be instructed in natural language to predict the most relevant text snippet, given an image, without directly optimizing for the task, similarly to the zero-shot capabilities of GPT-2 and 3. We found CLIP matches the performance of the original ResNet50 on ImageNet “zero-shot” without using any of the original 1.28M labeled examples, overcoming several major challenges in computer vision.

In [ ]:
#First, we load the respective CLIP model
model = SentenceTransformer("clip-ViT-B-32-multilingual-v1")

## Load dataset
Here we are trying to get some pictures from zalando that have both a picture and captions. 

In [ ]:
# Next, we get about 25k images from Unsplash 
img_folder = 'photos/'
if not os.path.exists(img_folder) or len(os.listdir(img_folder)) == 0:
    os.makedirs(img_folder, exist_ok=True)
    
    photo_filename = 'unsplash-25k-photos.zip'
    if not os.path.exists(photo_filename):   #Download dataset if does not exist
        util.http_get('http://sbert.net/datasets/'+photo_filename, photo_filename)
        
    #Extract all images
    with zipfile.ZipFile(photo_filename, 'r') as zf:
        for member in tqdm(zf.infolist(), desc='Extracting'):
            zf.extract(member, img_folder)
        

In [ ]:
import pinecone

# Connect to pinecone environment
pinecone.init(
    api_key="YOUR_API_KEY",
    environment="YOUR_ENV"  # find next to API key
)

index_name = 'gif-search'

# check if the gif-search exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=384,
        metric="cosine"
    )

# Connect to gif-search index we created
index = pinecone.Index(index_name)